# Convert code to AST and dump to file

In [23]:
%pip install astor
%pip install codegen

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for codegen: filename=codegen-1.0-py3-none-any.whl size=4722 sha256=d7a0a530861b167eac2a0bb6dd5edde892251486d4050d616fdd201dc1e729d4
  Stored in directory: c:\users\jess\appdata\local\pip\cache\wheels\08\1a\97\d47567b6121c4ba0b1f9d4a8e83e1a3fec2bf8b3c3ce43faf7
Successfully built codegen
Note: you may need to restart the kernel to use updated packages.


In [56]:
import ast
import astor 
import codegen 
import json
import ast
from graphviz import Digraph


In [78]:

# Read the Python code from a file
input_file = 'input.py'
with open(input_file, 'r') as file:
    code = file.read()

# Parse the code to create an AST
ast_tree = ast.parse(code)

# Change the ast code to json
ast_json = ast.dump(ast_tree)
output_file = 'ast_json.json'
with open(output_file, 'w') as file:
    file.write(ast_json)

# Dump the AST into a string with indentation for readability and save it to a file
ast_dump = ast.dump(ast_tree, indent=4)
# output_file = 'ast_dump.txt'
# with open(output_file, 'w') as file:
#     file.write(ast_dump)

    
regenerated_code = astor.to_source(ast_tree)

# Write the regenerated code back to a Python file
with open('regenerated_code.py', 'w') as file:
    file.write(code)


In [79]:
# Create a Graphviz Digraph object
dot = Digraph()

# Define a function to recursively add nodes to the Digraph
def add_node(node, parent=None):
    node_name = str(node.__class__.__name__)
    dot.node(str(id(node)), node_name)
    if parent:
        dot.edge(str(id(parent)), str(id(node)))
    for child in ast.iter_child_nodes(node):
        add_node(child, node)

# Add nodes to the Digraph
add_node(ast_tree)

# Render the Digraph as a PNG file
# def render_dot(dot):
dot.format = 'png'
dot.render('my_ast', view=True)


'my_ast.png'

In [75]:
# Create a Graphviz Digraph object
dot = Digraph()

# Define a function to recursively add nodes to the Digraph
def add_node(node, parent=None):
    node_name = str(node.__class__.__name__)
    dot.node(str(id(node)), node_name)
    if parent:
        dot.edge(str(id(parent)), str(id(node)))
    for child in ast.iter_child_nodes(node):
        add_node(child, node)


# Render the Digraph as a PNG file
def render_dot(dot):
	dot.format = 'png'
	dot.render('my_ast', view=True)


In [74]:
add_node(ast_tree)
render_dot(Digraph())

# Read AST from .txt and Convert back to code

In [22]:
class ASTRebuilder(ast.NodeTransformer):
    def visit_Str(self, node):
        return ast.Constant(value=node.s)  # Updated for Python 3.8 and later
    
    def visit_Constant(self, node):  # Handle constants
        return ast.Constant(value=node.value)

    def visit_Expr(self, node):
        return ast.Expr(value=self.visit(node.value))

    def visit_Module(self, node):
        return ast.Module(body=[self.visit(stmt) for stmt in node.body], type_ignores=[])

    def visit_FunctionDef(self, node):
        return ast.FunctionDef(
            name=node.name,
            args=self.visit(node.args),
            body=[self.visit(stmt) for stmt in node.body],
            decorator_list=[self.visit(decorator) for decorator in node.decorator_list],
            returns=node.returns,
            type_comment=node.type_comment
        )

    def visit_arguments(self, node):
        return ast.arguments(
            posonlyargs=[self.visit(arg) for arg in node.posonlyargs],
            args=[self.visit(arg) for arg in node.args],
            vararg=node.vararg,
            kwonlyargs=[self.visit(arg) for arg in node.kwonlyargs],
            kw_defaults=[self.visit(kw_default) for kw_default in node.kw_defaults],
            kwarg=node.kwarg,
            defaults=[self.visit(default) for default in node.defaults]
        )

    def visit_arg(self, node):
        return ast.arg(arg=node.arg, annotation=node.annotation, type_comment=node.type_comment)

    def visit_Call(self, node):
        return ast.Call(
            func=self.visit(node.func),
            args=[self.visit(arg) for arg in node.args],
            keywords=[self.visit(keyword) for keyword in node.keywords]
        )

    def visit_Assign(self, node):
        return ast.Assign(
            targets=[self.visit(target) for target in node.targets],
            value=self.visit(node.value),
            type_comment=node.type_comment
        )

    def visit_While(self, node):
        return ast.While(
            test=self.visit(node.test),
            body=[self.visit(stmt) for stmt in node.body],
            orelse=[self.visit(stmt) for stmt in node.orelse]
        )

    def visit_Compare(self, node):
        return ast.Compare(
            left=self.visit(node.left),
            ops=[self.visit(op) for op in node.ops],
            comparators=[self.visit(comp) for comp in node.comparators]
        )

    def visit_BinOp(self, node):
        return ast.BinOp(
            left=self.visit(node.left),
            op=self.visit(node.op),
            right=self.visit(node.right)
        )

    def visit_Name(self, node):
        return ast.Name(id=node.id, ctx=node.ctx)

    def visit_Attribute(self, node):
        return ast.Attribute(value=self.visit(node.value), attr=node.attr, ctx=node.ctx)

    def visit_Index(self, node):
        return ast.Index(value=self.visit(node.value))

    def visit_Load(self, node):
        return ast.Load()

    def visit_Store(self, node):
        return ast.Store()

    def visit_keyword(self, node):
        return ast.keyword(arg=node.arg, value=self.visit(node.value))

# Step 1: Read the AST dump from the file
with open('ast_dump.txt', 'r') as file:
    ast_dump = file.read()

# Step 2: Parse the AST dump string to get an AST of the AST dump
parsed_ast = ast.parse(ast_dump)

# Step 3: Reconstruct the AST object from the parsed AST
rebuilder = ASTRebuilder()
original_ast = rebuilder.visit(parsed_ast)

# Step 4: Convert the AST object back to Python code
if hasattr(ast, 'unparse'):
    code = ast.unparse(original_ast)
else:
    raise RuntimeError("ast.unparse is not available in this version of Python")

# Print the regenerated code
print(code)

# Optionally, write the regenerated code back to a Python file
with open('regenerated_code.py', 'w') as file:
    file.write(code)


Module(body=[Import(names=[alias(name='math')]), FunctionDef(name='foo', args=arguments(posonlyargs=[], args=[], kwonlyargs=[], kw_defaults=[], defaults=[]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Constant(value='hello world')], keywords=[]))], decorator_list=[]), Expr(value=Call(func=Name(id='foo', ctx=Load()), args=[], keywords=[])), Assign(targets=[Name(id='x', ctx=Store())], value=Constant(value=5)), While(test=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0)]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[])), Assign(targets=[Name(id='x', ctx=Store())], value=BinOp(left=Name(id='x', ctx=Load()), op=Sub(), right=Constant(value=1)))], orelse=[])], type_ignores=[])


In [38]:
expr = """
def foo():
   print("hello world")
x = 5
"""

# Parse the initial code to get an AST
p = ast.parse(expr)

# Serialize the AST to a string representation
ast_dump = ast.dump(p)

# Save the serialized AST to a file
with open('ast_dump.txt', 'w') as file:
    file.write(ast_dump)

In [43]:
# Read the AST dump from the file
with open('ast_dump.txt', 'r') as file:
    ast_dump = file.read()

# Function to recursively build AST nodes
def build_ast_node(node):
    if isinstance(node, dict):
        node_type = node['type']
        if node_type == 'Module':
            return ast.Module(body=[build_ast_node(child) for child in node['body']], type_ignores=node['type_ignores'])
        elif node_type == 'FunctionDef':
            return ast.FunctionDef(
                name=node['name'],
                args=build_ast_node(node['args']),
                body=[build_ast_node(child) for child in node['body']],
                decorator_list=node['decorator_list']
            )
        elif node_type == 'Expr':
            return ast.Expr(value=build_ast_node(node['value']))
        elif node_type == 'Call':
            return ast.Call(
                func=build_ast_node(node['func']),
                args=[build_ast_node(arg) for arg in node['args']],
                keywords=node['keywords']
            )
        elif node_type == 'Name':
            return ast.Name(id=node['id'], ctx=build_ast_node(node['ctx']))
        elif node_type == 'Load':
            return ast.Load()
        elif node_type == 'Store':
            return ast.Store()
        elif node_type == 'Constant':
            return ast.Constant(value=node['value'])
        elif node_type == 'Assign':
            return ast.Assign(
                targets=[build_ast_node(target) for target in node['targets']],
                value=build_ast_node(node['value'])
            )
        else:
            raise ValueError(f"Unsupported node type: {node_type}")
    else:
        raise ValueError("Node must be a dictionary")

# Build the AST from the structured representation
parsed_ast = build_ast_node(ast_dump)


# Parse the AST string to get an AST object
# parsed_ast = ast.parse(ast_dump, mode='eval')

# Convert the AST object to Python code using astor.to_source
code = astor.to_source(parsed_ast)

# Print the code using astor.to_source
print(code)

ValueError: Node must be a dictionary